In [3]:
import pandas as pd 

In [9]:
stocks = pd.read_csv("../../data/stocks.csv")

In [10]:
stocks.head()

,Date,ID,Close,High,Low,Open
0,1976-01-05,ARNC,"5,7790","5,7965","5,4802","5,4802"
1,1976-01-05,GT,"11,2790","11,2790","11,0298","11,0298"
2,1976-01-06,ARNC,"5,9369","5,9545","5,7965","5,7965"
3,1976-01-06,GT,"11,4660","11,4660","11,3413","11,3413"
4,1976-01-07,ARNC,"6,0072","6,0774","5,9019","5,9369"


In [11]:
stocks.loc[:, "Date"] = pd.to_datetime(stocks.Date, format="%Y-%m-%d")

In [19]:
stock_tickers = set(stocks.ID.unique())

In [31]:
len(stock_tickers)

1956

In [33]:
stocks.head()

,Date,ID,Close,High,Low,Open
0,1976-01-05 00:00:00,ARNC,"5,7790","5,7965","5,4802","5,4802"
1,1976-01-05 00:00:00,GT,"11,2790","11,2790","11,0298","11,0298"
2,1976-01-06 00:00:00,ARNC,"5,9369","5,9545","5,7965","5,7965"
3,1976-01-06 00:00:00,GT,"11,4660","11,4660","11,3413","11,3413"
4,1976-01-07 00:00:00,ARNC,"6,0072","6,0774","5,9019","5,9369"


In [4]:
stories = pd.read_csv("../../data/stories.csv")

In [5]:
stories.loc[:, "time"] = pd.to_datetime(stories.time)

In [20]:
stories_tickers = set(stories.stocks.unique())

In [21]:
stock_tickers.intersection(stories_tickers)

{'LVO'}